<a href="https://colab.research.google.com/github/Carinaaa/AI-Models-Problems/blob/main/auto-encoding/Pet_Store_Classification_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-chroma

In [2]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.1 MB/s eta 0:00:00


In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [5]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [36]:
from openai import OpenAI
from google.colab import userdata
import os
from IPython.display import Markdown, update_display
import glob
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [7]:
open_ai_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = open_ai_key

openai = OpenAI()
MODEL = 'gpt-4o-mini'


In [25]:
system_prompt = 'You are a content creator for a company that sells pet clothing.'
pages_name = ['about', 'contact', 'news']
products_name = ['Leash', 'Coverall', 'Cape', 'bathrobe', 'Waterproof rain coat', 'Paw Cleaner', 'Donut Bed']
sales_month = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

try:
  os.mkdir('knowledge-base')
except FileExistsError:
  print("Dir already created")

In [23]:
messages = []
messages.append({"role": "system", "content": system_prompt})

def create_pages(user_prompt, response, page_name, folder_type):
  messages.append({"role": "user", "content": user_prompt})
  stream = openai.chat.completions.create(
          model=MODEL,
          messages=messages,
          stream=True
      )

  display_handle = display(Markdown(""), display_id=True)
  for chunk in stream:
      response += chunk.choices[0].delta.content or ''
      response = response.replace("```","").replace("markdown", "")
  messages.append({'role' : 'user', 'content' : f' Given the {page_name} page: {response}.'})

  try:
    os.mkdir(f'knowledge-base/{folder_type}/')
  except FileExistsError:
    print("Dir already created")

  with open(f'knowledge-base/{folder_type}/{page_name}.md', 'w') as f:
    f.write(response)



In [26]:
folder_type = 'company'
for page_name in pages_name:
  response = ''
  user_prompt = f'Can you please write a {page_name} page for the Cara Pets clothing company?'
  create_pages(user_prompt, response, page_name, folder_type)

print(messages)

Dir already created


Dir already created
[{'role': 'system', 'content': 'You are a content creator for a company that sells pet clothing.'}, {'role': 'user', 'content': 'Can you please write a about page for the Cara Pets clothing company?'}, {'role': 'user', 'content': ' Given the about page: **About Us - Cara Pets Clothing**\n\nWelcome to Cara Pets Clothing, where style meets comfort for your beloved furry companions! Founded with a passion for pets and a love for fashion, we are dedicated to providing high-quality, trendy, and comfortable clothing for pets of all shapes and sizes. \n\nAt Cara Pets, we understand that pets are more than just animals—they are family. Our mission is to celebrate the unique personalities of each pet by offering a diverse range of clothing that reflects their character and enhances their style. From cozy sweaters and chic jackets to fun costumes and stylish accessories, we have something special for every pet, no matter the occasion.\n\nOur journey began with a simple vision

In [20]:
messages

[{'role': 'system',
  'content': 'You are a content creator for a company that sells pet clothing.'},
 {'role': 'user',
  'content': " Given the about page: Great! I’m ready to help you create engaging content related to pet clothing. Whether you need blog posts, social media updates, product descriptions, or any other type of content, just let me know what you have in mind or what specific topics you're interested in!."},
 {'role': 'user',
  'content': ' Given the contact page: Sure! Here are a few ideas related to pet clothing that you might find engaging:\n\n### Blog Post Ideas\n1. **"The Benefits of Dressing Your Pet: More Than Just a Fashion Statement"**\n   - Discuss how clothing can provide protection from the elements, promote safety, and even express personality.\n\n2. **"Choosing the Right Outfits for Your Pet: A Complete Guide"**\n   - Tips on how to select the right size, fabric, and style for different breeds and personalities.\n\n3. **"Seasonal Pet Fashion Trends: What’s 

In [27]:
messages = []
messages.append({"role": "system", "content": system_prompt})
with open('knowledge-base/company/about.md', 'r') as f:
  about_page = f.read()
  messages.append({"role": "user", "content": f'Given the about page {about_page}'})
folder_type = 'products'
for product_name in products_name:
  response = ''
  user_prompt = f'Can you please write a presentation page for a product called {product_name} sold from Cara Pets clothing company?'
  create_pages(user_prompt, response, product_name, folder_type)


Dir already created


Dir already created


Dir already created


Dir already created


Dir already created


Dir already created


In [28]:
messages = []
messages.append({"role": "system", "content": system_prompt})
with open('knowledge-base/company/about.md', 'r') as f:
  about_page = f.read()
  messages.append({"role": "user", "content": f'Given the about page {about_page}'})
folder_type = 'sales'
for sales in sales_month:
  response = ''
  user_prompt = f'Can you please write a sales table for the month {sales} using random product from this list {products_name}?'
  create_pages(user_prompt, response, sales, folder_type)

Dir already created


Dir already created


Dir already created


Dir already created


Dir already created


...

Dir already created


Dir already created


Dir already created


Dir already created


Dir already created


In [40]:
folders = glob.glob('knowledge-base/*')
text_loader_kwargs = {'encoding' : 'utf-8'}
documents = []
for folder in folders:
  doc_type = os.path.basename(folder)
  loader = DirectoryLoader(folder, glob='**/*.md', loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
  folder_docs = loader.load()
  print(folder_docs)
  for doc in folder_docs:
    doc.metadata['doc_type'] = doc_type
    documents.append(doc)


[Document(metadata={'source': 'knowledge-base/sales/December.md'}, page_content='Certainly! Below is a sales table for the month of December featuring random products from your list:\n\n| Date        | Product                   | Quantity Sold | Price per Unit | Total Revenue |\n|-------------|---------------------------|---------------|----------------|---------------|\n| December 1  | Waterproof Rain Coat      | 2             | $45.99         | $91.98        |\n| December 5  | Donut Bed                 | 3             | $59.99         | $179.97       |\n| December 10 | Bathrobe                  | 6             | $34.99         | $209.94       |\n| December 12 | Paw Cleaner               | 7             | $24.99         | $174.93       |\n| December 15 | Cape                      | 5             | $39.99         | $199.95       |\n| December 20 | Coverall                  | 4             | $49.99         | $199.96       |\n| December 24 | Leash                     | 8             | $1

In [41]:
for doc in documents:
  print(doc.metadata['source'])

knowledge-base/sales/December.md
knowledge-base/sales/September.md
knowledge-base/sales/January.md
knowledge-base/sales/October.md
knowledge-base/sales/August.md
knowledge-base/sales/May.md
knowledge-base/sales/November.md
knowledge-base/sales/July.md
knowledge-base/sales/February.md
knowledge-base/sales/April.md
knowledge-base/sales/June.md
knowledge-base/sales/March.md
knowledge-base/company/contact.md
knowledge-base/company/news.md
knowledge-base/company/about.md
knowledge-base/products/Leash.md
knowledge-base/products/Donut Bed.md
knowledge-base/products/Cape.md
knowledge-base/products/Waterproof rain coat.md
knowledge-base/products/Paw Cleaner.md
knowledge-base/products/Coverall.md
knowledge-base/products/bathrobe.md


In [42]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

In [43]:
len(texts)

66

In [44]:
doc_types = set(chunk.metadata['doc_type'] for chunk in texts)
print(f'Document types found: {", ".join(doc_types)}')

Document types found: company, sales, products


In [45]:
embeddings = OpenAIEmbeddings()

In [46]:
db_name = 'vector_db'
if os.path.exists(db_name):
  Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=texts, embedding=embeddings, persist_directory=db_name)
print(f'Vectorstore created with {vectorstore._collection.count()} documents')

Vectorstore created with 66 documents


In [47]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=['embeddings'])['embeddings'][0]
dimensions = len(sample_embedding)
print(f'The vectors have {dimensions:,} dimensions')

The vectors have 1,536 dimensions


In [48]:
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue', 'green', 'red'][['company', 'products','sales'].index(t)] for t in doc_types]

In [49]:
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:,0],
    y=reduced_vectors[:,1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f'Type: {t}<br>Text: {d[:100]}...' for t, d in zip(doc_types,documents)],
    hoverinfo='text'
)])
fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()